In [1]:
import pyspark

In [2]:
import os

try:
    os.remove("metastore_db/db.lck")
    os.remove("metastore_db/dbex.lck")
except:
    pass


from pyspark.sql import SparkSession

def build_spark_session(app_name, memory='4g', executors=4):
    return SparkSession.builder\
                      .appName(app_name)\
                      .config('spark.executor.memory', memory)\
                      .config('spark.executor.instances', executors)\
                      .getOrCreate()

spark_session = build_spark_session(app_name='ok-google')

from pyspark.sql import functions as f



1 - joindre les dataframe sur la clé ID_CLIENT en concervant tous les clients du périmètre.
2 - compter le nombre de ID_CLIENT et vérifier qu'il correspond aux nombre d'ID_CLIENT dans la variable perimètre.
3 - Caster les variables continues en double et sauvergarder alors le df obtenu dans le repertoire data sur le cluster.
4 - Pour les variables catégorielles, créer une nouvelle variable qui prend la modalité de la variable courante si elle existe et "NA" sinon.
5- Verifier la cohérence des variables continue. Par exemple pour une variable comme age mettre à -1 tous les ages <0 ou>120ans

In [3]:
perimetre = spark_session.read.csv("/home/galatawan/Bureau/spark_doc/data_clients/sample_perimetre.csv", header=True)
histo_client_raw = spark_session.read.csv("/home/galatawan/Bureau/spark_doc/data_clients/sample_histo_client.csv", header=True)
histo_train_raw = spark_session.read.csv("/home/galatawan/Bureau/spark_doc/data_clients/sample_histo_train.csv", header=True)
histo_lowcost_raw = spark_session.read.csv("/home/galatawan/Bureau/spark_doc/data_clients/sample_histo_lowcost.csv", header=True)
visites_raw = spark_session.read.csv("/home/galatawan/Bureau/spark_doc/data_clients/sample_visites.csv", header=True)

1/lire les fichiers de données

In [9]:
## ecrire une fonction pour transformer les features quantitatives ("anciennete", "recence_cmd", "AGE", etc..) en float
histo_client_raw.head()

Row(ID_CLIENT='000843db32fbaecfbb047ca0bb04b1f9f4d9425a', anciennete='1550', recence_cmd='36', AGE=None, LBL_STATUT_CLT='Grand', LBL_GEO_AIR='Aéroports de Paris Orly', LBL_GRP_SEGMENT_NL='Spectateur', LBL_SEG_COMPORTEMENTAL='Chasseurs Bons Plans', LBL_GEO_TRAIN='Paris', LBL_SEGMENT_ANTICIPATION='Mixte', FLG_CMD_CARTE_1225='0')

In [10]:
histo_train_raw.head()

Row(ID_CLIENT='000843db32fbaecfbb047ca0bb04b1f9f4d9425a', nb_od='1', mean_nb_passagers='1.0', mean_duree_voyage='274.6666666666667', mean_mt_voyage='58.666666666666664', mean_tarif_loisir='0.0', mean_classe_1='0.0', mean_pointe='0.0', mean_depart_we='0.0')

In [11]:
histo_lowcost_raw.head()

Row(ID_CLIENT='003fb9dca8de374386d0fa97b570950583111931', flg_cmd_lowcost='1', flg_track_nl_lowcost='0', flg_track_nl='1')

In [12]:
visites_raw.head()

Row(ID_CLIENT='000843db32fbaecfbb047ca0bb04b1f9f4d9425a', days_since_last_visit='8', tx_conversion='0.1111111111111111')

In [13]:
histo_client_raw.describe()

DataFrame[summary: string, ID_CLIENT: string, anciennete: string, recence_cmd: string, AGE: string, LBL_STATUT_CLT: string, LBL_GEO_AIR: string, LBL_GRP_SEGMENT_NL: string, LBL_SEG_COMPORTEMENTAL: string, LBL_GEO_TRAIN: string, LBL_SEGMENT_ANTICIPATION: string, FLG_CMD_CARTE_1225: string]

2 - identifier les variables continues et transformer leurs modalités en double.

In [4]:

client_cols_to_keep = ["ID_CLIENT", 'LBL_STATUT_CLT','LBL_GEO_AIR',
            'LBL_SEG_COMPORTEMENTAL','LBL_GEO_TRAIN','LBL_GRP_SEGMENT_NL',
            'LBL_SEGMENT_ANTICIPATION','FLG_CMD_CARTE_1225']

def cast_columns_of_df(df, cols_to_cast, col_to_keep, cast_type='double'):
    """cast continuous columns into double since all columns are """
    return df.select(col_to_keep + [(df[feature].cast('double'))
                    for feature in cols_to_cast if 'ID_CLIENT' not in feature])

histo_train = cast_columns_of_df(histo_train_raw, histo_train_raw.columns,
                                 ["ID_CLIENT"], cast_type='double')
histo_lowcost = cast_columns_of_df(histo_lowcost_raw, histo_lowcost_raw.columns,
                                 ["ID_CLIENT"], cast_type='double')

visites = cast_columns_of_df(visites_raw, visites_raw.columns,
                             ["ID_CLIENT"], cast_type='double')

histo_client = cast_columns_of_df(histo_client_raw,
                                  ["anciennete", "recence_cmd", "AGE"],
                                  client_cols_to_keep,
                                 cast_type='double')


3 - joindre les differentes sources de données en se basant sur les données du périmètre
(tous les individus du périmèetre devront apparaitre dans la jointure avec des valeurs NULL 
 si nécessaire pour les colonnes en provenance d'autres sources).
                                                                                          

In [5]:
df=perimetre.join(histo_train,on='ID_CLIENT',how='left'
                 ).join(histo_lowcost, on='ID_CLIENT', how='left'
                       ).join(visites, on='ID_CLIENT', how='left'
                             ).join(histo_client, on='ID_CLIENT', how='left')


In [9]:
df.toPandas().head()

,ID_CLIENT,nb_od,mean_nb_passagers,mean_duree_voyage,mean_mt_voyage,mean_tarif_loisir,mean_classe_1,mean_pointe,mean_depart_we,flg_cmd_lowcost,...,LBL_STATUT_CLT,LBL_GEO_AIR,LBL_SEG_COMPORTEMENTAL,LBL_GEO_TRAIN,LBL_GRP_SEGMENT_NL,LBL_SEGMENT_ANTICIPATION,FLG_CMD_CARTE_1225,anciennete,recence_cmd,AGE
0,000843db32fbaecfbb047ca0bb04b1f9f4d9425a,1.0,1.0,274.666667,58.666667,0.0,0.0,0.00,0.00,NaN,...,Grand,Aéroports de Paris Orly,Chasseurs Bons Plans,Paris,Spectateur,Mixte,0,1550.0,36.0,NaN
1,001338752ea32d9de129c8f8bdf3e2224cf0bd71,1.0,1.0,232.000000,128.200000,1.0,1.0,0.00,0.00,NaN,...,Grand,Aéroport de Marseille Provence (MRS),Comportement Pro,Marseille,Spectateur,Anticipateur,0,1667.0,25.0,35.0
2,003fb9dca8de374386d0fa97b570950583111931,3.0,1.5,248.250000,46.475000,0.5,0.0,0.25,0.25,1.0,...,Moyen moins,Aéroport de Lyon - Saint Exupéry,Rythmes scolaires,Lyon,Spectateur,Peu Anticipateur,1,395.0,15.0,25.0
3,004efa6652e570ef68944b780738e159fbf2aeb5,1.0,1.0,117.000000,35.500000,1.0,0.5,0.00,0.50,NaN,...,Moyen moins,Aéroport de Lyon - Saint Exupéry,Sans contraintes,Lyon,Spectateur,Tres Anticipateur,0,2188.0,20.0,31.0
4,005dd0b718a8f4598ae3044e60f9c20222eb3a35,1.0,1.0,84.000000,65.500000,1.0,0.0,0.50,0.50,NaN,...,Petit,Aéroport de Strasbourg,Sans contraintes,Metz,Endormi,Non Anticipateur,0,3005.0,15.0,32.0


In [18]:
df.printSchema()

root
 |-- ID_CLIENT: string (nullable = true)
 |-- geo_train: string (nullable = true)
 |-- geo_air: string (nullable = true)
 |-- cc_jeunes: string (nullable = false)
 |-- segt_rfm: string (nullable = true)
 |-- segt_anticipation: string (nullable = true)
 |-- segt_comportemental: string (nullable = true)
 |-- segt_nl: string (nullable = true)
 |-- age: double (nullable = true)
 |-- recence_cmd: double (nullable = true)
 |-- mean_duree_voyage: double (nullable = true)
 |-- recence_visite: double (nullable = true)
 |-- mean_mt_voyage: double (nullable = true)
 |-- anciennete: double (nullable = true)
 |-- nb_od: double (nullable = true)
 |-- mean_nb_passagers: double (nullable = true)
 |-- mean_tarif_loisir: double (nullable = true)
 |-- mean_classe_1: double (nullable = true)
 |-- mean_pointe: double (nullable = true)
 |-- mean_depart_we: double (nullable = true)
 |-- tx_conversion: double (nullable = true)
 |-- flg_cmd_lowcost: string (nullable = false)
 |-- flg_track_nl_lowcost: str

faire une jointure entre les informations des différentes tables.
NB: on conservera tous les clients de la table perimetre.
    En effet, ce sont les cleints qu'on souhaite scorer

combien a t'on de features quantitatives, qualitatives 
remplacer les valeurs manquantes

In [10]:
dfeat = df.dtypes
quanti = 0
quali = 0
var_quali = []
var_quanti = []
for i in dfeat:
    if i[1] == "double":
        quanti += 1
        var_quanti.append(i[0])
    else:
        quali += 1
        var_quali.append(i[0])

print("features quantitatives = {} \n features qualitatives = {}".format(quanti,quali))#le nombre de var quanti et quali

features quantitatives = 16 
 features qualitatives = 8


enlever la variable a prédire 'flg_cmd_lowcost' dans le lot des variables quantitave et le mettre dans le lot des quali

In [18]:
#var_quanti.remove('flg_cmd_lowcost')
var_quali.append('flg_cmd_lowcost')


In [19]:
var_quali

['ID_CLIENT',
 'LBL_STATUT_CLT',
 'LBL_GEO_AIR',
 'LBL_SEG_COMPORTEMENTAL',
 'LBL_GEO_TRAIN',
 'LBL_GRP_SEGMENT_NL',
 'LBL_SEGMENT_ANTICIPATION',
 'FLG_CMD_CARTE_1225',
 'flg_cmd_lowcost']

In [20]:
def replace_missing_values(df, var_quanti, var_quali):
    moy_quanti = [df.select(f.mean(feature)).collect()[0][0] for feature in var_quanti]
    replace_quali = df.select([f.when(df[feature].isNotNull(),df[feature])\
                               .otherwise(-1).alias(feature) for i,feature in enumerate(var_quali)]\
                             +[f.when(df[feature].isNotNull(),df[fature])\
                              .otherwise(moy_quanti[i]).alias(feature) for i,fature in enumerate(continuous)])
    return replace_quali

quelles sont les differentes modalites de la feature LBL_STATUT_CLT

quelles sont les features avec valeurs manquantes
remplacer les valeurs manquantes par -1 pour toutes les features

In [6]:
def input_df(df):
    ds = df.select('ID_CLIENT',
    f.when(df.LBL_GEO_TRAIN.isin(['Toulouse', 'Lille', 'Dijon',
                                  'Lyon', 'Marseille', 'Paris',
                                  'Nice', 'Limoges','Rouen','Rennes',
                                  'Montpellier', 'Bordeaux', 'Metz',
                                  'Strasbourg']), df.LBL_GEO_TRAIN)\
               .otherwise('na').alias('geo_train'),
    f.when(df.LBL_GEO_AIR.isin(['Aéroports de Paris Orly',
                                'Aéroport de Bâle-Mulhouse / Bassel',
                                'Aéroport Lille Lesquin', 'Aéroport de Rennes',
                                'Aéroport de Nantes Atlantique',
                                'Aéroport de Marseille Provence  (MRS)', 
                                'Aéroport de Bordeaux Mérignac',
                                'Aéroports de Paris Roissy-Charles-de Gaulle', 
                                "Aéroport de Nice Côte d'Azur",
                                'Aéroport de Strasbourg',
                                'Aéroport de Lyon - Saint Exupéry', 
                                'Aéroport de Toulouse Blagnac']), df.LBL_GEO_AIR)\
               .otherwise('na').alias('geo_air'),
    f.when(df.FLG_CMD_CARTE_1225 == '1', '1')\
                   .otherwise('0').alias('cc_jeunes'),
    f.when(df.LBL_STATUT_CLT.isin(['Tres grand', 'Nouveau actif',
                                   'Moyen moins', ' Prospect', ' Petit',
                                   'Inactif', 'Tres petit',
                                   'Nouveau prospect', 'Moyen plus',
                                   'Grand']), df.LBL_STATUT_CLT)\
                   .otherwise('na').alias('segt_rfm'),
    f.when(df.LBL_SEGMENT_ANTICIPATION.isin(['Peu Anticipateur', 'Tres Anticipateur',
                                             'Anticipateur', 'Mixte', 'Non Anticipateur',
                                             'Non Defini']), df.LBL_SEGMENT_ANTICIPATION)\
                   .otherwise('na').alias('segt_anticipation'),
    f.when(df.LBL_SEG_COMPORTEMENTAL.isin(['Mono-commande',
                                           'Comportement Pro',
                                           'Exclusifs Agence', 
                                           'Anticipateurs Methodiques',
                                           'Chasseurs Bons Plans', 
                                           'Rythmes scolaires', 'Nouveaux',
                                           'Sans contraintes']),
           df.LBL_SEG_COMPORTEMENTAL).otherwise('na').alias('segt_comportemental'), 
    f.when(df.LBL_GRP_SEGMENT_NL.isin(['Endormi', 'Spectateur', 'Acteur',
                                       'Eteint', 'Non defini']),
           df.LBL_GRP_SEGMENT_NL).otherwise('na').alias('segt_nl'),
    f.when(((df.AGE > 0) & (df.AGE < 100)), df.AGE)\
                   .otherwise(-1).alias('age'),
    f.when(df.recence_cmd >= 0, df.recence_cmd)\
                   .otherwise(-1).alias('recence_cmd'),
    f.when(((df.mean_duree_voyage > 0) & (df.mean_duree_voyage < 750)),
           df.mean_duree_voyage).otherwise(-1).alias('mean_duree_voyage'),
    f.when(df.days_since_last_visit >= 0, df.days_since_last_visit)\
                   .otherwise(-1).alias('recence_visite'),
    f.when(df.mean_mt_voyage > 0, df.mean_mt_voyage)\
                   .otherwise(-1).alias('mean_mt_voyage'),
    f.when(df.anciennete >= 0, df.anciennete)\
                   .otherwise(-1).alias('anciennete'),
    f.when(df.nb_od > 0, df.nb_od)\
                   .otherwise(-1).alias('nb_od'),
    f.when(df.mean_nb_passagers > 0, df.mean_nb_passagers)\
                   .otherwise(-1).alias('mean_nb_passagers'),
    f.when(df.mean_tarif_loisir >= 0, df.mean_tarif_loisir)\
                   .otherwise(-1).alias('mean_tarif_loisir'),
    f.when(df.mean_classe_1 >= 0, df.mean_classe_1)\
                   .otherwise(-1).alias('mean_classe_1'),
    f.when(df.mean_pointe >= 0, df.mean_pointe)\
                   .otherwise(-1).alias('mean_pointe'),
    f.when(df.mean_depart_we >= 0, df.mean_depart_we)\
                   .otherwise(-1).alias('mean_depart_we'),
    f.when(df.tx_conversion >= 0, df.tx_conversion)\
                   .otherwise(-1).alias('tx_conversion'),
    f.when(df.flg_cmd_lowcost == 1, '1')\
                   .otherwise('0').alias('flg_cmd_lowcost'),
    f.when(df.flg_track_nl_lowcost == 1, '1')\
                   .otherwise('0').alias('flg_track_nl_lowcost'), 
    f.when(df.flg_track_nl == 1, '1')\
                   .otherwise('0').alias('flg_track_nl'))
    
    return ds
df = input_df(df)

In [31]:
df.toPandas()

,ID_CLIENT,geo_train,geo_air,cc_jeunes,segt_rfm,segt_anticipation,segt_comportemental,segt_nl,age,recence_cmd,...,nb_od,mean_nb_passagers,mean_tarif_loisir,mean_classe_1,mean_pointe,mean_depart_we,tx_conversion,flg_cmd_lowcost,flg_track_nl_lowcost,flg_track_nl
0,000843db32fbaecfbb047ca0bb04b1f9f4d9425a,Paris,Aéroports de Paris Orly,0,Grand,Mixte,Chasseurs Bons Plans,Spectateur,-1.0,36.0,...,1.0,1.0,0.0,0.0,0.00,0.000000,0.111111,0,0,0
1,001338752ea32d9de129c8f8bdf3e2224cf0bd71,Marseille,Aéroport de Marseille Provence (MRS),0,Grand,Anticipateur,Comportement Pro,Spectateur,35.0,25.0,...,1.0,1.0,1.0,1.0,0.00,0.000000,0.130435,0,0,0
2,003fb9dca8de374386d0fa97b570950583111931,Lyon,Aéroport de Lyon - Saint Exupéry,1,Moyen moins,Peu Anticipateur,Rythmes scolaires,Spectateur,25.0,15.0,...,3.0,1.5,0.5,0.0,0.25,0.250000,1.000000,1,0,1
3,004efa6652e570ef68944b780738e159fbf2aeb5,Lyon,Aéroport de Lyon - Saint Exupéry,0,Moyen moins,Tres Anticipateur,Sans contraintes,Spectateur,31.0,20.0,...,1.0,1.0,1.0,0.5,0.00,0.500000,0.125000,0,0,0
4,005dd0b718a8f4598ae3044e60f9c20222eb3a35,Metz,Aéroport de Strasbourg,0,na,Non Anticipateur,Sans contraintes,Endormi,32.0,15.0,...,1.0,1.0,1.0,0.0,0.50,0.500000,0.333333,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084212,ffdfe078a2e824b19b577130a160680e1e4cce75,Rennes,Aéroport de Nantes Atlantique,1,Moyen plus,Peu Anticipateur,Sans contraintes,Spectateur,24.0,38.0,...,2.0,1.0,0.2,0.0,0.80,0.000000,0.400000,0,0,0
1084213,ffe870a88fe3929188c0259d56d6aa5cfaff29df,Marseille,Aéroport de Marseille Provence (MRS),0,Moyen moins,Peu Anticipateur,Rythmes scolaires,Non defini,51.0,39.0,...,1.0,1.0,0.0,0.0,0.00,0.500000,0.285714,1,0,0
1084214,fff435283c3573b60f40e06c91aa7a5653134458,na,na,0,Moyen plus,Peu Anticipateur,Chasseurs Bons Plans,Non defini,61.0,3.0,...,1.0,2.0,1.0,0.0,0.50,0.000000,0.333333,0,0,0
1084215,fff8e3138cc4122bcf15366b7c6893a0b57fa943,Rennes,Aéroport de Nantes Atlantique,1,Tres grand,Mixte,Sans contraintes,Spectateur,23.0,3.0,...,1.0,1.0,0.0,0.0,1.00,0.428571,0.170213,0,0,0


Quelles sont les differentes valeurs de notre label : flg_cmd_lowcost

In [10]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression


##### features engineering et modélisation

In [12]:

def preprocessed_df(df, label="flg_cmd_lowcostIndex"):
    max_values_to_define_str_cols = 10
    id_col = 'ID_CLIENT'
    
    dty = dict(df.dtypes)
    str_cols = [k for k, v in dty.items() if v == 'string']
    str_cols.remove(id_col)
    
    for c in str_cols:
        stringIndexer = StringIndexer(inputCol=c, outputCol=c+"Index")
        model_str = stringIndexer.fit(df)
        df = model_str.transform(df).drop(c)

    input_cols = df.columns
    input_cols.remove(id_col)
    input_cols.remove(label)
    
    assembler = VectorAssembler(inputCols=input_cols,
                            outputCol="features")
    df = assembler.transform(df)
    
    featureIndexer = VectorIndexer(inputCol="features", 
                   outputCol="indexedFeatures", 
                   maxCategories=max_values_to_define_str_cols).fit(df)
    return featureIndexer.transform(df), df


data, dff = preprocessed_df(df)

data1=data.sample(False, 0.1, 42)

In [16]:
data1.toPandas()

,ID_CLIENT,age,recence_cmd,mean_duree_voyage,recence_visite,mean_mt_voyage,anciennete,nb_od,mean_nb_passagers,mean_tarif_loisir,...,cc_jeunesIndex,segt_rfmIndex,segt_anticipationIndex,segt_comportementalIndex,segt_nlIndex,flg_cmd_lowcostIndex,flg_track_nl_lowcostIndex,flg_track_nlIndex,features,indexedFeatures
0,00660b90838bcbcc223018a3b0ccedefa94c34a1,21.0,31.0,67.00,29.0,8.00,153.0,1.0,1.0,0.0,...,0.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,"[21.0, 31.0, 67.0, 29.0, 8.0, 153.0, 1.0, 1.0,...","[21.0, 31.0, 67.0, 29.0, 8.0, 153.0, 1.0, 1.0,..."
1,00b2a46ecd1c941fabdd8074ba29af412d581348,61.0,22.0,234.75,3.0,74.00,397.0,3.0,1.0,0.0,...,0.0,5.0,0.0,5.0,4.0,0.0,0.0,0.0,"[61.0, 22.0, 234.75, 3.0, 74.0, 397.0, 3.0, 1....","[61.0, 22.0, 234.75, 3.0, 74.0, 397.0, 3.0, 1...."
2,00bb5b917b2a0689e16300ecab32a24330b86a81,36.0,11.0,122.50,1.0,28.60,679.0,1.0,2.0,-1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,"[36.0, 11.0, 122.5, 1.0, 28.6, 679.0, 1.0, 2.0...","[36.0, 11.0, 122.5, 1.0, 28.6, 679.0, 1.0, 2.0..."
3,010acf8023ef067cb84ad6b6e92bc011355f0a77,-1.0,11.0,46.50,11.0,7.80,4374.0,1.0,2.0,0.0,...,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,"(-1.0, 11.0, 46.5, 11.0, 7.800000000000001, 43...","(-1.0, 11.0, 46.5, 11.0, 7.800000000000001, 43..."
4,01b94b68daa31121133cddce1f5526701ea368e0,-1.0,77.0,226.50,-1.0,50.00,4567.0,1.0,1.0,0.0,...,0.0,6.0,3.0,4.0,0.0,0.0,0.0,0.0,"(-1.0, 77.0, 226.5, -1.0, 50.0, 4567.0, 1.0, 1...","(-1.0, 77.0, 226.5, -1.0, 50.0, 4567.0, 1.0, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108454,feb296f89ccd4d27605bde3612a3aaead5dd9f01,35.0,62.0,95.00,61.0,67.00,62.0,1.0,1.0,1.0,...,0.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,"(35.0, 62.0, 95.0, 61.0, 67.0, 62.0, 1.0, 1.0,...","(35.0, 62.0, 95.0, 61.0, 67.0, 62.0, 1.0, 1.0,..."
108455,fefb55fe92fe595959fc54a4b5f8f198bf6aef6d,54.0,50.0,215.00,8.0,92.65,68.0,1.0,2.0,1.0,...,0.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,"[54.0, 50.0, 215.0, 8.0, 92.65, 68.0, 1.0, 2.0...","[54.0, 50.0, 215.0, 8.0, 92.65, 68.0, 1.0, 2.0..."
108456,ff1cc7999994145bec8e8b9105aaba8df3040253,19.0,41.0,213.00,38.0,66.50,970.0,1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,"(19.0, 41.0, 213.0, 38.0, 66.5, 970.0, 1.0, 1....","(19.0, 41.0, 213.0, 38.0, 66.5, 970.0, 1.0, 1...."
108457,ffb3d6dd4c4b9b55543f05c60be61af9a80b79de,42.0,24.0,99.00,12.0,22.00,122.0,1.0,1.0,1.0,...,0.0,3.0,6.0,3.0,0.0,1.0,1.0,1.0,"[42.0, 24.0, 99.0, 12.0, 22.0, 122.0, 1.0, 1.0...","[42.0, 24.0, 99.0, 12.0, 22.0, 122.0, 1.0, 1.0..."


prelever un sample de data pour notre modelisation

#### Logistic regression

quelle est le label est renseigne pour la modelisation?

In [19]:

lr = LogisticRegression(labelCol="", 
                        featuresCol="indexedFeatures",elasticNetParam=0.5)


In [ ]:
Ajuster le modele de regression logistique et calculer les coefficients de notre modele.

Evaluer votre modele courbe roc, precision, rappel, etc....

predire alors les clients lowcoast sur un sample de data n'ayant pas servi à l'apprentissage

##### Random Forest

In [20]:
classifier = RandomForestClassifier(labelCol="flg_cmd_lowcostIndex", 
                                    featuresCol="indexedFeatures",
                                    maxDepth=15, numTrees=100)

model_rf = classifier.fit(data1)

NameError: name 'data1' is not defined

evaluer les performance de notre modele

In [42]:
# une autre technique pour evaluer le modele

from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

Evaluer soit meme le score en calculant le nombre de VP, FP, VN et FN
on calculera alors le score qui VP+VN/VP+VN+FP+FN
nb: la prediction est automatiquement creee dans le data set et correspond à la colonne prediction

In [21]:
calculer egalement le rappel et la precision

SyntaxError: invalid syntax (<ipython-input-21-96ad390423ec>, line 1)

In [ ]:
from pyspark.ml.linalg import Vectors
scoreAndLabels = map(lambda x: (Vectors.dense([1.0 - x[0], x[0]]), x[1]),
                     [(0.1, 0.0), (0.1, 1.0), (0.4, 0.0), (0.6, 0.0), (0.6, 1.0), (0.6, 1.0), (0.8, 1.0)])
dataset = spark_session.createDataFrame(scoreAndLabels, ["raw", "label"])
dataset.head()

#### how to create a spark data frame

In [62]:
from pyspark.sql import Row
import datetime

l = [(datetime.date(2018,1,3), 'Ankit',25),
     (datetime.date(2018,2,3), 'Jalfaizy',22),
     (datetime.date(2018,1,5), 'saurabh',20),
     (datetime.date(2018,1,12), 'Bala',26)]
rdd = spark_session.sparkContext.parallelize(l)
people = rdd.map(lambda x: Row(date=x[0], name=x[1], age=int(x[2])))
schemaPeople = spark_session.createDataFrame(people)
schemaPeople.toPandas()

,age,date,name
0,25,2018-01-03,Ankit
1,22,2018-02-03,Jalfaizy
2,20,2018-01-05,saurabh
3,26,2018-01-12,Bala


1- compter le nombre de personne totale 
2- compter le nombre de fille et de garcon
3- quel est l'age moyen, median mini et maxi dans chaque groupe (garcon, fille)
4 - quelle est la date de dernière visite de chaque client par rapport à la date d'aujourd'hui (la colonne date correspond à la date de visite)
5 - quels sont les personnes qui ont une date de visite < 400 jours.

In [63]:
dd = schemaPeople.select("*",
                         f.lit(datetime.date.today()).alias("date_max"))
dd.select("*", f.datediff('date_max', 'date')\
                    .alias('days_since_last_visit')).toPandas()

,age,date,name,date_max,days_since_last_visit
0,25,2018-01-03,Ankit,2018-02-21,49
1,22,2018-02-03,Jalfaizy,2018-02-21,18
2,20,2018-01-05,saurabh,2018-02-21,47
3,26,2018-01-12,Bala,2018-02-21,40
